In [2]:
import numpy as np
import firedrake as fd


Discretization

$$
H_i^n = \eta(x_i, t_n) = \eta(x_0 + \sum_{j=0}^{i-1}k_j, t_0 + n\Delta t), \quad k_i = x_{i+1} - x_i
$$

$$
\Phi_i^n = \phi(x_i, t_n)
$$

Stencils:


$$
\frac{\partial \eta}{\partial t} = \frac{H^{n+1}_i - H^n_i}{\Delta t} + \mathcal{O}(\Delta t^2)
$$

$$
\frac{\partial \eta}{\partial x} = \frac{H^n_{i+1} - H^n_i}{k_i} + \mathcal{O}(k_i^2)
$$

$$
\frac{\partial \phi}{\partial t} = \frac{\Phi^{n+1}_i - \Phi^n_i}{\Delta t} + \mathcal{O}(\Delta t^2)
$$

$$
\frac{\partial \phi}{\partial x} = \frac{\Phi^n_{i+1} - \Phi^n_i}{k_i} + \mathcal{O}(k_i^2)
$$

Schemes:

$$
H^{n+1}_i = H^n_i + \Delta t \left(\frac{H^n_{i+1} - H^n_i}{k_i}\tilde{U}_i^n + \tilde{W}^n_i + \tilde{W^n_i} \left[\frac{H^n_{i+1} - H^n_i}{k_i}\right]^2 \right)
$$

$$
\tilde{\Phi}_{i}^{n+1} = \tilde{\Phi}_{i}^{n} + \Delta t\left(-gH_{i}^{n} - \frac{1}{2}\left( \left(\tilde{U}_{i}^{n} \right)^{2}-(\tilde{W}_{i}^{n})^{2}
\left( 1+\left( \frac{H_{i+1}^{n}-H_{i}^{n}}{k_{i}}\right)^{2} \right)\right)\right).
$$

In [ ]:
def update_eta(self, fs_coords: np.ndarray, dt: float = 1e-2) -> np.ndarray:
    """
    Updates eta in pseudo-time given first order foward Euler scheme.
    
    Parameters
    ----------
    eta : np.ndarray
        The current free surface elevation.
    dt : float
        Pseudo-time step.
    """
    fs_coords.copy() # Copy the array to avoid modifying the original

    # Sort the array by x-coordinates to ensure fs_coords = [[x_0, z_0], [x_1, z_1], ... [x_M-1, z_M-1]]
    fs_coords = fs_coords[fs_coords[:, 0].argsort()] 

    x = fs_coords[:, 0]
    eta = fs_coords[:, 1]

    # Compute the x-distance between all nodes
    dx = x[1:] - x[:-1] #dx = x_{i+1} - x_{i}

    # First order x-stencil
    eta_x = (eta[1:] - eta[:-1])/dx

    fs_velocity = self.velocity.at(fs_coords[:-1,:])
    Un = fs_velocity[:, 0]
    Wn = fs_velocity[:, 1]

    # Compute eta pseudo-time step
    eta_new = eta[:-1] + dt * (eta_x * Un + Wn + Wn*eta_x**2) 
    return eta_new


In [ ]:
def update_phi(self, fs_coords: np.ndarray, dt: float = 1e-2) -> np.ndarray:
    """
    Updates phi in pseudo-time given first order foward Euler scheme.
    
    Parameters
    ----------
    phi : np.ndarray
        The current free surface elevation.
    dt : float
        Pseudo-time step.
    """
    fs_coords.copy() # Copy the array to avoid modifying the original

    # Sort the array by x-coordinates to ensure fs_coords = [[x_0, z_0], [x_1, z_1], ... [x_M-1, z_M-1]]
    fs_coords = fs_coords[fs_coords[:, 0].argsort()] 

    x = fs_coords[:, 0]
    eta = fs_coords[:, 1]

    # Compute the x-distance between all nodes
    dx = x[1:] - x[:-1] #dx = x_{i+1} - x_{i}

    # First order x-stencil for eta_x
    eta_x = (eta[1:] - eta[:-1])/dx

    phi = self.VelocityPotential.at(fs_coords[:-1,:])
    fs_velocity = self.velocity.at(fs_coords[:-1,:])
    Un = fs_velocity[:, 0]
    Wn = fs_velocity[:, 1]
    

    # Compute eta pseudo-time step
    g = 9.81
    phi_new = phi + dt*(-g*eta[:-1] - 0.5*(Un**2 + Wn**2*(1 + eta_x**2)))
    return phi_new